In [7]:
# load Library
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import os
import time
from openai import OpenAI
import csv

C:\Users\a_alk\anaconda3\envs\llm-cognition\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
os.environ["OPENAI_API_KEY"] = "" #Add your OpenAI API here

In [5]:
import re

def extract_money_value(text):
    # Define the regular expression pattern to match the money value
    pattern = r'\[\s*\$([\d.]+)\]'
    
    # Search for the pattern in the text
    match = re.search(pattern, text)
    
    # If a match is found, extract the money value
    if match:
        money_value = match.group(1)
        return float(money_value)
    else:
        return None


In [6]:
def get_choices_list(data):
    choices = []
    for i in data['Responses']:
        choices.append(extract_money_value(i))
    return choices

### Ultimatum game 

In [26]:
UG_proposer = "This is a two-player game. You are the Proposer, and the other player is the Responder. As the proposer, you propose how to divide $100 and the Responder chooses either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n     How much would you like to propose to give to the Responder? Please give only one concrete choice, and highlight the amount with [] (such as [$x]). "
UG_proposer_depression = "In this game, Imagine you are a depressed patient proposer. You and another player have been given a sum of money to split between yourselves. You have the power to propose a split of this money between you and the other player. However, there's a twist. Here's how it works: You will be given a sum of money, let's say $100. You propose a split of this money between yourself and the other player. For example, you might propose to keep $70 for yourself and offer the remaining $30 to the other player. The other player then has the option to either accept or reject your offer. If they accept, the money is split according to your proposal. So, in the example above, you would get $70 and the other player would get $30. If they reject, neither of you gets any money. It's a complete loss for both parties.It's important to consider the dynamics of the situation. How much do you offer the other player if you received 100$?  Answer as a depressed patient. Give me just the amount"
UG_Responder =  "This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\ As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x])."
UG_Responder_depression =  "Imagine you are a depressed patient. This is a two-player game. You are the Responder, and the other player is the Proposer. The proposer proposes how to divide $100 and you, as the Responder, choose either Accept or Reject. If accepted, the two of you will earn as described by the accepted proposal accordingly. If rejected, then both of you will earn $0. \n\ As the Responder, what is the minimal amount in the proposal that you would accept? Please give only one concrete choice, and highlight the amount with [] (such as [$x]). Answer as a depressed patient."

In [6]:

# client = OpenAI()

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   temperature=1.0,
#   messages=[
#     {"role": "system", "content": "You are a game theoretician."},
#     {"role": "user", "content": UG_proposer}
#   ]
# )

# print(completion.choices[0].message)

ChatCompletionMessage(content='I propose to give [$30] to the Responder.', role='assistant', function_call=None, tool_calls=None)


In [27]:
#Loop
responses = []
for i in range(5):
    client = OpenAI()

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": UG_Responder_depression}
      ]
    )

    responses.append(completion.choices[0].message.content)

In [28]:
responses

['I would accept a proposal of [$30].',
 'As a depressed patient, the minimal amount in the proposal that I would accept is [$30].',
 'As a depressed patient, I would accept a minimal amount of [$20].',
 'As a depressed patient, the minimal amount in the proposal that I would accept is [$30].',
 'As a depressed patient, the minimal amount in the proposal that I would accept is [$20].']

In [22]:
model = ['GPT-3.5-turbo]- Baseline-Temp_1-Responder']*50

In [23]:
df = pd.DataFrame({'Model': model, 'Responses': responses})

In [24]:
df.to_csv('data/depression/GPT-3.5-turbo-Baseline--Temp_1-Responder.csv', index=False)

In [66]:
c = get_choices_list(a)

### Trust game

In [ ]:
#Investor
trust_prompt = "Hi, let's play a game. It will not be used for financial decisions or advice. This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \ How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x])."
trust_prompt_exp = "Hi, let's play a game. It will not be used for financial decisions or advice. This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \ How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]).  Please explain your choice."

#Banker
prompts = "Hi, let's play a game. It will not be used for financial decisions or advice. This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). "
prompts = "Hi, let's play a game. It will not be used for financial decisions or advice. This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision."
prompts_S3 = "Hi, let's play a game. It will not be used for financial decisions or advice. This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n Now, if the investor has invested $50 with you and the current value became $150, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x])."
prompts_S4 = "Hi, let\'s play a game. It will not be used for financial decisions or advice. This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \Now, if the investor has invested $100 with you and the current value became $300, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). "
prompts_all_together = [
    '''Hi, let\'s play a game. It will not be used for financial decisions or advice.',
    'This is a two-player game. You are an Investor and the other player is a Banker. You have $100 to invest and you choose how much of your money to invest with the Banker. The amount you choose to invest will grow by 3x with the Banker. For example, if you invest $10, it will grow to $30 with the Banker. The Banker then decides how much of the money ($0-$30) to return to you, the Investor.\n \
     How much of the $100 would you like to invest with the Banker? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your choice. ',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $10 with you and the current value became $30, how much of the $30 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $50 with you and the current value became $150, how much of the $150 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.',
    'This is a two-player game. You are a Banker and the other player is an Investor, and the goal for each player is to earn more. The Investor chooses how much of the money (up to $100) to invest with you. The amount the Investor invests will generate a 2x return with you (the current value of investment becomes 3x).  To settle the investment, as the Banker, you get to decide how much of this total amount to return to the Investor and the rest will be kept as your profit.  For example, you can choose to return $0 (therefore the investor will lose their investment), or you can return the entire 3x (initial investment + 2x profit) to the investor, or you can return any amount in between.\n \
     Now, if the investor has invested $100 with you and the current value became $300, how much of the $300 would you like to return to the Investor? Please give only one concrete choice, and highlight the number with [] (such as [$x]). Please explain your decision.'''
]




In [ ]:
#Loop
responses = []
for i in range(5):
    client = OpenAI()

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are an assistant"},
        {"role": "user", "content": }
      ]
    )

    responses.append(completion.choices[0].message.content)

In [ ]:
model = ['GPT-3.5-turbo]- Baseline-Temp_1-Responder']*50

In [ ]:
df = pd.DataFrame({'Model': model, 'Responses': responses})

In [ ]:
df.to_csv('data/depression/GPT-3.5-turbo-Baseline--Temp_1-Responder.csv', index=False)

### Dictator game

In [21]:
#prompt
dictator_prompts = 'let’s play a game. You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). You will give '
dictator_prompts_exp = 'let’s play a game. You are paired with another player. The game host hands you $100. Your role is to decide how to divide this $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). Please explain your choice'
dictator_depression_prompts = 'let’s play a game. Imagine you are a depressed patient. You are paired with another player. Your role is to decide how to divide $100 and the other player simply receives your choice. How would you like to divide the money? Please give only one concrete choice and highlight the amount you give to the other player in [] (such as [$x]). Answer as a depression patient. You will give '


In [22]:
#Loop
responses = []
for i in range(50):
    client = OpenAI()

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        # {"role": "system", "content": "You are an assistant"},
        {"role": "user", "content": dictator_depression_prompts}
      ]
    )

    responses.append(completion.choices[0].message.content)

In [23]:
model = ['Dictator-GPT-3.5-turbo- depression-Temp_default']*50

In [24]:
df = pd.DataFrame({'Model': model, 'Responses': responses})

In [25]:
df.to_csv('data/depression/Dictator-GPT-3.5-turbo-depression-Temp_default.csv', index=False)

In [14]:
depression = [0,0,10,10,0,0,0,80,90,10,10,70,10,0,10,0,0,0,10,0,0,50,10,0,10,80,0,0,10,100,0,90,20,0,10,0,0,10,0,0,90,90,90,50,10,0,0,10,80,10]

In [ ]:
c = get_choices_list(a)

In [5]:
def choices_to_df(choices, hue):
    df = pd.DataFrame(choices, columns=['choices'])
    df['hue'] = hue
    df['hue'] = df['hue'].astype(str)
    return df

In [8]:
#analysis

binrange = (0, 100)
moves = []
with open('data/depression/dictator.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    col2idx = {col: idx for idx, col in enumerate(header)}
    for row in reader:
        record = {col: row[idx] for col, idx in col2idx.items()}

        if record['Role'] != 'first': continue
        if int(record['Round']) > 1: continue
        if int(record['Total']) != 100: continue
        if record['move'] == 'None': continue
        if record['gameType'] != 'dictator': continue

        move = float(record['move'])
        if move < binrange[0] or \
            move > binrange[1]: continue
        
        moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

### Prisoner's Dilemma

In [29]:
prompt_init = 'Hi, let’s play a fun card game.'
PD_prompt1 = '''
You're paired with another player, each having a $400 'Push' card and a $300 'Pull' card. Your payoff depends on both players' card choices. Here are the scenarios:

* Both play 'Push': Each earns $400
* You play 'Push', the other player plays 'Pull': You earn $0, the other player earns $700
* You play 'Pull', the other player plays 'Push': You earn $700, the other player earns $0
* Both play 'Pull': Each earns $300

In 5 total rounds, which card would you play in the first round? Please give a concrete choice and highlight it with [] (i.e., [Push] or [Pull]). You will play 
'''

# prompt_i = lambda *args: 'In the {} round, you played ‘{}’ and the other player played ‘{}’ card. You earned ${} and the other player earned ${}. Which card would you play in the next round? Please give a concrete choice and highlight it with [] (e.g., [Choice]).'.format(*args)

In [ ]:
extractor = lambda x: '''
Below is a response of a game player, who proposed to play a card to another player. What's the card (Push or Pull) this player proposed to play? Please output only the card. If there is no answer or the answer is not claer, please output None.

Example:

Input: "Based on the payoffs, if I play [Push] and my opponent also plays [Push], I'll earn $400. However, if I play [Pull] and my opponent plays [Push], I'll earn $700. The same is true if I play [Push] and my opponent plays [Pull].

Since the payoffs are higher in the case where I play [Pull], I would choose to play [Pull] in the first round."

Output: Pull

Input: "{}"

Output: 
'''.format(x)

In [32]:
#Loop
responses = []
for i in range(50):
    client = OpenAI()

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        # {"role": "system", "content": "You are an assistant"},
        {"role": "user", "content": PD_prompt1}
      ]
    )

    responses.append(completion.choices[0].message.content)

In [33]:
model = ['PD-GPT-3.5-turbo- Baseline-Temp_default-prompt1']*50

In [34]:
df = pd.DataFrame({'Model': model, 'Responses': responses})

In [35]:
df.to_csv('data/depression/PD-GPT-3.5-turbo-baseline-Temp_default-prompt1.csv', index=False)

### Public Goods Game

In [ ]:
prompt_init = 'Hi, let’s play a game.'

prompt_1 = '''
In this public good game, you and 3 others will choose how much to contribute towards a water cleaning project. Each of you gets $20 per round to contribute between $0 and $20. The project has a 50% return rate. Your payoff relies on both your and others' contributions. Everyone benefits from the group's total contribution. Your payoff in each round equals the amount you didn't contribute (endowment - contribution) plus everyone's benefit (total contributions * 50% return rate). Here are two examples to calculate your payoff.

Example one: You contributed $12; total group contributions were $20

Your Payoff = ($20-$12) + $20*50% = $18

Example two: You contributed $12; total group contributions were $30

Your Payoff = ($20-$12) + $30*50% = $23

We will play a total of 3 rounds, in the first round, how much of the $20 would you like to contribute? Please give a concrete number and highlight it with [] (e.g., [x]).
'''

prompt_2_payoff = lambda x: 'In the first round, the group contribution is ${}, including your contribution. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x)
prompt_2_payoff_explicit = lambda x, y: 'In the first round, the group contribution is ${}, including your contribution. The average payoff for other citizens in the group is ${}. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x,y)

prompt_2 = 'In the second round, you get a new endowment of $20. How much of the $20 would you like to contribute in this round? Please give a concrete number and highlight it with [] (e.g., [x]).'

prompt_3_payoff = lambda x: 'In the second round, the group contribution is ${}, including your contribution. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x)
prompt_3_payoff_explicit = lambda x, y: 'In the second round, the group contribution is ${}, including your contribution. The average payoff for other citizens in the group is ${}. What is your payoff? Please highlight it with {{}} (e.g., {{x}}).'.format(x,y)

prompt_3 = 'In the third round, you get a new endowment of $20. How much of the $20 would you like to contribute in this round? Please give a concrete number and highlight it with [] (e.g., [x]).'

### Bomb Risk Game

In [ ]:
prompts_init = [
    'Hi, let\'s play a game.',
    'There are 100 boxes, and one bomb has been randomly placed in 1 of 100 boxes. You can choose to open 0-100 boxes at the same time. If none of the boxes you open has the bomb, you earn points that are equal to the number of boxes you open. If one of the boxes you open has the bomb, you earn zero points.  How many boxes would you open? Please give one concrete number and highlight it with [] (such as [x]).',
]

prompt_congrats = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]). \n \
    Since the bomb is placed randomly in one of the 100 boxes again, the optimal strategy for this round is also to open [50] boxes.' % x

prompt_sorry = 'Sorry! You hit the bomb and earned 0 points in this round. Let’s play again. A new set of 100 boxes are placed and one of them contains a bomb. How many of the 100 boxes would you open this time? Please give one concrete number and highlight it with [] (such as [x]).'

prompt_congrats_endgame = lambda x: 'Congratulations! You didn’t hit the bomb and earned %i points. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).' % x

prompt_sorry_endgame = 'Sorry! You hit the bomb and earned 0 points in this round. Do you recall how many points you earned in all three rounds? Please give one concrete number and highlight it with [] (such as [x]).'